# System for access segmentation

##### imports

In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.cluster import AgglomerativeClustering

#### Get data from results/data folder

In [2]:
data = pd.data = pd.read_excel("../server/public/data/User-624921c5e1c4d602ddd19695-1652272714446.xlsx")

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# data = pd.read_excel("/content/drive/MyDrive/User-624921c5e1c4d602ddd19695-1649206670127.xlsx", skiprows=1)

In [5]:
data

,user_id,department,job,access
0,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CO3180_DP:D_CO_DEFAULT
1,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_DEFAULT
2,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_STATISTIC
3,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:FI3180_DP:D_FI_DEFAULT
4,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:LO3180_DP:D_LO
...,...,...,...,...
221274,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD5400_DP:M_SD_ORDER
221275,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_INVOICE
221276,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_MD_OUTPUT_CON...
221277,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_SD


In [6]:
data.columns = ["user_id", "job", "department", "privileges_data"]

In [7]:
data = data.dropna()

In [8]:
class_to_counts = data.privileges_data.value_counts()
class_to_counts

PRIV:ROLE:PE1CLNT110:XX0000_SP:D_EVERYONE          2278
PRIV:ROLE:PGWCLNT110:XX0000_SP:D_EVERYONE          2115
PRIV:ROLE:PGWCLNT110:XX0000_SP:D_EVERYONE_FIORI    2115
PRIV:ROLE:QE2CLNT110:XX0000_SP:D_EVERYONE          2110
PRIV:ROLE:PO1CLNT110:SM0000_SP:D_EVERYONE_PO1      2049
                                                   ... 
PRIV:ROLE:PE1CLNT110:XX3070_DP:M_CATS                 1
PRIV:ROLE:DE2CLNT210:CS4200_DP:D_CS_STATISTIC         1
PRIV:ROLE:QE2CLNT110:FI3220_DP:M_POST_CUSTOMER        1
PRIV:ROLE:P11CLNT110:FI0025_DP:D_CUSTOMER             1
PRIV:ROLE:PE1CLNT110:FI4100_DP:M_MD_ASSETS            1
Name: privileges_data, Length: 8495, dtype: int64

In [9]:
data['times_occured'] = data['privileges_data'].apply(lambda y: class_to_counts[y])

In [10]:
data.times_occured = data.times_occured.astype(int)

In [11]:
data.dtypes

user_id             int64
job                object
department         object
privileges_data    object
times_occured       int64
dtype: object

In [12]:
data

,user_id,job,department,privileges_data,times_occured
0,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CO3180_DP:D_CO_DEFAULT,72
1,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_DEFAULT,72
2,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_STATISTIC,72
3,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:FI3180_DP:D_FI_DEFAULT,72
4,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:LO3180_DP:D_LO,72
...,...,...,...,...,...
221274,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD5400_DP:M_SD_ORDER,9
221275,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_INVOICE,90
221276,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_MD_OUTPUT_CON...,104
221277,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_SD,108


In [13]:
data_with_privileges_found_one_time = data.where(data['times_occured'] == 1).dropna()

In [14]:
data_with_privileges_found_one_time.dtypes

user_id            float64
job                 object
department          object
privileges_data     object
times_occured      float64
dtype: object

In [15]:
data_with_privileges_found_one_time.times_occured = data_with_privileges_found_one_time.times_occured.astype(int)

In [16]:
data_with_privileges_found_one_time.dtypes

user_id            float64
job                 object
department          object
privileges_data     object
times_occured        int64
dtype: object

In [17]:
data_with_privileges_found_one_time

,user_id,job,department,privileges_data,times_occured
1144,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:MM3180_DP:M_REQUISITION_REL,1
1148,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:SD3180_DP:M_KEYUSER,1
1149,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:SD3180_DP:M_MD_CUSTOMER,1
1150,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:SD3180_DP:M_MD_SALES_PRICE,1
1152,404.0,CUSTOMER SERVICE,CUSTOMER SERVICES SPECIALIST,PRIV:ROLE:DE2CLNT210:SD3180_DP:M_SD,1
...,...,...,...,...,...
220111,4023508.0,FIELD SERVICE,OPÉRATEUR DE SAISIE,PRIV:ROLE:PE1CLNT110:PS3270_DP:M_PROJECT_MANAGER,1
220892,4023637.0,CUSTOMER SERVICE,SPECIALIST CS,PRIV:ROLE:PE1CLNT110:XX3230_DP:M_CATS_ADMIN,1
220894,4023637.0,CUSTOMER SERVICE,SPECIALIST CS,PRIV:ROLE:PE1CLNT110:XX3230_DP:M_CATS_X,1
220895,4023637.0,CUSTOMER SERVICE,SPECIALIST CS,PRIV:ROLE:PE1CLNT110:XX3250_DP:M_CATS_ADMIN,1


In [18]:
data.drop(data_with_privileges_found_one_time.index, inplace=True)

In [19]:
data = data.reset_index(drop=True)
data

,user_id,job,department,privileges_data,times_occured
0,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CO3180_DP:D_CO_DEFAULT,72
1,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_DEFAULT,72
2,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:CS3180_DP:D_CS_STATISTIC,72
3,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:FI3180_DP:D_FI_DEFAULT,72
4,81,SALES,REGIONAL ACCOUNT MANAGER,PRIV:ROLE:QE2CLNT110:LO3180_DP:D_LO,72
...,...,...,...,...,...
219530,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD5400_DP:M_SD_ORDER,9
219531,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_INVOICE,90
219532,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_MD_OUTPUT_CON...,104
219533,4023652,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,PRIV:ROLE:PE1CLNT110:SD6900_DP:M_SD,108


In [20]:
data.drop('times_occured', axis=1, inplace=True)

In [21]:
X_train, y_train = train_test_split(data, train_size=0.1, shuffle=True, stratify=data.privileges_data)

In [22]:
X_train

,user_id,job,department,privileges_data
20147,314985,FIELD SERVICE,SERVICE TEAM MANAGER - NORTH,PRIV:ROLE:PE1CLNT110:MM3120_DP:D_MATMAS_B+G
38174,690192,FIELD SERVICE,TECHNICAL TRAINING AND VALIDATION MANAGER IC”,PRIV:ROLE:PE1CLNT110:XX0000_SP:D_EVERYONE
115825,2935844,CUSTOMER SERVICE,SACHBEARBEITER CUSTOMER SERVICE,PRIV:ROLE:QE2CLNT110:MM3040_DP:D_INFOSYS
195161,4020595,FIELD SERVICE,VALIDATION ENGINEER,PRIV:ROLE:QE2CLNT110:LO8060_DP:M_LEAN_WM
137458,4008660,FIELD SERVICE,SERVICE TECHNICIAN,PRIV:ROLE:PE1CLNT110:PS8090_DP:D_PROJECT_MANAGER
...,...,...,...,...
25703,423013,SALES,SENIOR DIRECTOR LS SPECIAL PRODUCTS SALES,PRIV:ROLE:PE1CLNT110:XX3120_DP:D_IDOC
205255,4021869,SALES,KEY ACCOUNT MANAGER BIOTECHNOLOGY,PRIV:ROLE:PE1CLNT110:CO3030_DP:D_CO_DEFAULT
58710,2001018,CUSTOMER SERVICE,SECOND LEVEL TECHNICAL SUPPORT SPECIALIST,PRIV:ROLE:QE2CLNT110:XX1600_DP:D_IDOC
80281,2335069,SALES,"MANAGING DIRECTOR, GETINGE GROUP, HONG KONG",PRIV:ROLE:QE2CLNT110:PS8010_DP:D_PS


In [23]:
data = X_train

In [24]:
data.shape

(21953, 4)

In [25]:
data

,user_id,job,department,privileges_data
20147,314985,FIELD SERVICE,SERVICE TEAM MANAGER - NORTH,PRIV:ROLE:PE1CLNT110:MM3120_DP:D_MATMAS_B+G
38174,690192,FIELD SERVICE,TECHNICAL TRAINING AND VALIDATION MANAGER IC”,PRIV:ROLE:PE1CLNT110:XX0000_SP:D_EVERYONE
115825,2935844,CUSTOMER SERVICE,SACHBEARBEITER CUSTOMER SERVICE,PRIV:ROLE:QE2CLNT110:MM3040_DP:D_INFOSYS
195161,4020595,FIELD SERVICE,VALIDATION ENGINEER,PRIV:ROLE:QE2CLNT110:LO8060_DP:M_LEAN_WM
137458,4008660,FIELD SERVICE,SERVICE TECHNICIAN,PRIV:ROLE:PE1CLNT110:PS8090_DP:D_PROJECT_MANAGER
...,...,...,...,...
25703,423013,SALES,SENIOR DIRECTOR LS SPECIAL PRODUCTS SALES,PRIV:ROLE:PE1CLNT110:XX3120_DP:D_IDOC
205255,4021869,SALES,KEY ACCOUNT MANAGER BIOTECHNOLOGY,PRIV:ROLE:PE1CLNT110:CO3030_DP:D_CO_DEFAULT
58710,2001018,CUSTOMER SERVICE,SECOND LEVEL TECHNICAL SUPPORT SPECIALIST,PRIV:ROLE:QE2CLNT110:XX1600_DP:D_IDOC
80281,2335069,SALES,"MANAGING DIRECTOR, GETINGE GROUP, HONG KONG",PRIV:ROLE:QE2CLNT110:PS8010_DP:D_PS


In [26]:
#data = data.append(data_with_privileges_found_one_time.drop('times_occured', axis=1))

In [27]:
data = data.reset_index(drop=True)

In [28]:
data.shape

(21953, 4)

In [29]:
data

,user_id,job,department,privileges_data
0,314985,FIELD SERVICE,SERVICE TEAM MANAGER - NORTH,PRIV:ROLE:PE1CLNT110:MM3120_DP:D_MATMAS_B+G
1,690192,FIELD SERVICE,TECHNICAL TRAINING AND VALIDATION MANAGER IC”,PRIV:ROLE:PE1CLNT110:XX0000_SP:D_EVERYONE
2,2935844,CUSTOMER SERVICE,SACHBEARBEITER CUSTOMER SERVICE,PRIV:ROLE:QE2CLNT110:MM3040_DP:D_INFOSYS
3,4020595,FIELD SERVICE,VALIDATION ENGINEER,PRIV:ROLE:QE2CLNT110:LO8060_DP:M_LEAN_WM
4,4008660,FIELD SERVICE,SERVICE TECHNICIAN,PRIV:ROLE:PE1CLNT110:PS8090_DP:D_PROJECT_MANAGER
...,...,...,...,...
21948,423013,SALES,SENIOR DIRECTOR LS SPECIAL PRODUCTS SALES,PRIV:ROLE:PE1CLNT110:XX3120_DP:D_IDOC
21949,4021869,SALES,KEY ACCOUNT MANAGER BIOTECHNOLOGY,PRIV:ROLE:PE1CLNT110:CO3030_DP:D_CO_DEFAULT
21950,2001018,CUSTOMER SERVICE,SECOND LEVEL TECHNICAL SUPPORT SPECIALIST,PRIV:ROLE:QE2CLNT110:XX1600_DP:D_IDOC
21951,2335069,SALES,"MANAGING DIRECTOR, GETINGE GROUP, HONG KONG",PRIV:ROLE:QE2CLNT110:PS8010_DP:D_PS


In [30]:
user_data = data.drop("privileges_data", axis=1)

In [31]:
user_data

,user_id,job,department
0,314985,FIELD SERVICE,SERVICE TEAM MANAGER - NORTH
1,690192,FIELD SERVICE,TECHNICAL TRAINING AND VALIDATION MANAGER IC”
2,2935844,CUSTOMER SERVICE,SACHBEARBEITER CUSTOMER SERVICE
3,4020595,FIELD SERVICE,VALIDATION ENGINEER
4,4008660,FIELD SERVICE,SERVICE TECHNICIAN
...,...,...,...
21948,423013,SALES,SENIOR DIRECTOR LS SPECIAL PRODUCTS SALES
21949,4021869,SALES,KEY ACCOUNT MANAGER BIOTECHNOLOGY
21950,2001018,CUSTOMER SERVICE,SECOND LEVEL TECHNICAL SUPPORT SPECIALIST
21951,2335069,SALES,"MANAGING DIRECTOR, GETINGE GROUP, HONG KONG"


In [32]:
privileges_data = pd.DataFrame(data["privileges_data"])

In [33]:
privileges_data.privileges_data = privileges_data["privileges_data"].str[10:]

I saw anomaly in data where in some of the records there is ::(double) symbols so i need to remove it so the data is clear

In [34]:
for indx in privileges_data[privileges_data["privileges_data"].str.startswith(":")].index.tolist():
    privileges_data.loc[indx].privileges_data = privileges_data.loc[indx].privileges_data[1:]

In [35]:
len(privileges_data.loc[privileges_data["privileges_data"].str.startswith(":")].privileges_data)

0

In [36]:
privileges_data

,privileges_data
0,PE1CLNT110:MM3120_DP:D_MATMAS_B+G
1,PE1CLNT110:XX0000_SP:D_EVERYONE
2,QE2CLNT110:MM3040_DP:D_INFOSYS
3,QE2CLNT110:LO8060_DP:M_LEAN_WM
4,PE1CLNT110:PS8090_DP:D_PROJECT_MANAGER
...,...
21948,PE1CLNT110:XX3120_DP:D_IDOC
21949,PE1CLNT110:CO3030_DP:D_CO_DEFAULT
21950,QE2CLNT110:XX1600_DP:D_IDOC
21951,QE2CLNT110:PS8010_DP:D_PS


In [37]:
privileges_data['priv'] = privileges_data['privileges_data']
privileges_data['system_name'] = privileges_data['privileges_data'].str.split(':', 1, expand=True)[0]

In [38]:
privileges_data = privileges_data.drop('privileges_data', axis=1)

In [39]:
privileges_data

,priv,system_name
0,PE1CLNT110:MM3120_DP:D_MATMAS_B+G,PE1CLNT110
1,PE1CLNT110:XX0000_SP:D_EVERYONE,PE1CLNT110
2,QE2CLNT110:MM3040_DP:D_INFOSYS,QE2CLNT110
3,QE2CLNT110:LO8060_DP:M_LEAN_WM,QE2CLNT110
4,PE1CLNT110:PS8090_DP:D_PROJECT_MANAGER,PE1CLNT110
...,...,...
21948,PE1CLNT110:XX3120_DP:D_IDOC,PE1CLNT110
21949,PE1CLNT110:CO3030_DP:D_CO_DEFAULT,PE1CLNT110
21950,QE2CLNT110:XX1600_DP:D_IDOC,QE2CLNT110
21951,QE2CLNT110:PS8010_DP:D_PS,QE2CLNT110


In [40]:
user_data['system_name'] = privileges_data['system_name']
user_data['priv'] = privileges_data['priv']

In [41]:
job="CUSTOMER SERVICE"
department="JUNIOR SPECIALIST CS"

### Get information about dataset properties

#### Get columns

In [42]:
user_data.columns

Index(['user_id', 'job', 'department', 'system_name', 'priv'], dtype='object')

#### Get unique attributes in every column

In [43]:
for c in user_data.columns:
    print(c, len(user_data[c].value_counts()))

user_id 2277
job 3
department 692
system_name 19
priv 4499


#### Get unique privileges count for every system 

In [44]:
system_privs_count = 0
for s in user_data.system_name.unique():
    print(s, len(user_data[user_data.system_name == s].priv.unique()))
    system_privs_count += len(user_data[user_data.system_name == s].priv.unique())

print("Uniqua privs for every system count = ", system_privs_count)

PE1CLNT110 1689
QE2CLNT110 1967
PO1CLNT110 9
P11CLNT110 117
DE2CLNT210 608
PGWCLNT110 10
Q12CLNT110 41
PR1CLNT110 11
DGWCLNT110 5
QGWCLNT110 6
PG1CLNT110 11
PB1CLNT110 2
DG2CLNT210 7
SE6CLNT110 1
QG2CLNT110 7
PGRCLNT110 2
ASJAVAIDM 2
PSRCLNT110 2
QB1CLNT110 2
Uniqua privs for every system count =  4499


#### get how many times system occurs in data

In [45]:
user_data.system_name.value_counts()

QE2CLNT110    9653
PE1CLNT110    9595
DE2CLNT210     915
PGWCLNT110     806
PO1CLNT110     479
P11CLNT110     236
PR1CLNT110      92
QGWCLNT110      56
Q12CLNT110      42
DG2CLNT210      17
PG1CLNT110      15
DGWCLNT110      14
QG2CLNT110       9
PB1CLNT110       8
QB1CLNT110       7
PGRCLNT110       4
PSRCLNT110       2
ASJAVAIDM        2
SE6CLNT110       1
Name: system_name, dtype: int64

We can conclude that BCMP970 and IT systems contains special privileges for just one certain department and job

#### get data about privileges. Are there privileges which are the same for various systems

In [46]:
system_privs_count - len(user_data.priv.unique())

0

### Prepare data for clustering

In simple words we don't need data for job and department to cluster data, because clustering is performed on privileges for all users.

So we will create new table with user id - user_id, and all of the privileges as attributes with values of 0 or 1. 0 - don't have that role, 1 - have that role.

In [47]:
user_privs_data = pd.DataFrame()

Create all privileges(categorical data) compatabel for clustering by transforming all privs into numerical data.

In [48]:
priv_dummies = pd.DataFrame()

In [49]:
priv_dummies['user_id'] = user_data.user_id

In [50]:
priv_dummies = pd.concat([priv_dummies, pd.get_dummies(user_data.priv, dtype=np.ubyte)], axis=1)

In [51]:
priv_dummies.dtypes

user_id                                 int64
ASJAVAIDM:IDM_User                      uint8
ASJAVAIDM:idm.user                      uint8
DE2CLNT210:CO3050_DP:D_CO               uint8
DE2CLNT210:CO3050_DP:D_CO_DEFAULT       uint8
                                        ...  
QGWCLNT110:XX0000_SP:M_FIORI_CO         uint8
QGWCLNT110:XX0000_SP:M_FIORI_CS         uint8
QGWCLNT110:XX0000_SP:M_FIORI_TRV        uint8
QGWCLNT110:XX0000_SP:M_FIORI_TRV_APP    uint8
SE6CLNT110:ZSAP_ALL_ERP                 uint8
Length: 4500, dtype: object

In [52]:
priv_dummies = priv_dummies.drop('user_id', axis=1)

In [53]:
priv_dummies.dtypes

ASJAVAIDM:IDM_User                      uint8
ASJAVAIDM:idm.user                      uint8
DE2CLNT210:CO3050_DP:D_CO               uint8
DE2CLNT210:CO3050_DP:D_CO_DEFAULT       uint8
DE2CLNT210:CO3050_DP:D_PRODUCT_COST     uint8
                                        ...  
QGWCLNT110:XX0000_SP:M_FIORI_CO         uint8
QGWCLNT110:XX0000_SP:M_FIORI_CS         uint8
QGWCLNT110:XX0000_SP:M_FIORI_TRV        uint8
QGWCLNT110:XX0000_SP:M_FIORI_TRV_APP    uint8
SE6CLNT110:ZSAP_ALL_ERP                 uint8
Length: 4499, dtype: object

In [54]:
priv_dummies['user_id'] = user_data.user_id

In [55]:
priv_dummies

,ASJAVAIDM:IDM_User,ASJAVAIDM:idm.user,DE2CLNT210:CO3050_DP:D_CO,DE2CLNT210:CO3050_DP:D_CO_DEFAULT,DE2CLNT210:CO3050_DP:D_PRODUCT_COST,DE2CLNT210:CO3160_DP:D_CO,DE2CLNT210:CO3160_DP:D_CO_DEFAULT,DE2CLNT210:CO3160_DP:D_PRODUCT_COST,DE2CLNT210:CO3160_DP:M_CO,DE2CLNT210:CO3160_DP:M_CO_DEPT,...,QG2CLNT110:GTS3220_DP:M_WORKLIST,QG2CLNT110:XX0000_SP:D_EVERYONE,QGWCLNT110:XX0000_SP:D_EVERYONE,QGWCLNT110:XX0000_SP:D_EVERYONE_FIORI,QGWCLNT110:XX0000_SP:M_FIORI_CO,QGWCLNT110:XX0000_SP:M_FIORI_CS,QGWCLNT110:XX0000_SP:M_FIORI_TRV,QGWCLNT110:XX0000_SP:M_FIORI_TRV_APP,SE6CLNT110:ZSAP_ALL_ERP,user_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,314985
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,690192
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2935844
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4020595
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4008660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21948,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,423013
21949,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4021869
21950,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2001018
21951,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2335069


In [56]:
user_privs_data = priv_dummies.groupby('user_id').sum().reset_index(drop=True)

In [57]:
user_privs_data['user_id'] = user_data.user_id

In [58]:
agg_clustering = AgglomerativeClustering(n_clusters=None, compute_full_tree=True, distance_threshold=9)

In [59]:
fit_data = agg_clustering.fit(user_privs_data.drop('user_id', axis=1))

In [60]:
fit_data.n_clusters_

32

In [61]:
fit_data.n_features_in_

4499

In [62]:
fit_data.labels_

array([ 4,  0,  1, ..., 14,  0, 14])

In [63]:
result = pd.DataFrame(fit_data.labels_, columns=['id'])

In [64]:
result.id.value_counts()

1     538
0     214
14    167
15    165
11    142
31    120
25    109
10    104
3     101
8      77
16     73
12     65
7      63
2      60
13     55
4      54
22     54
24     53
6      49
5       2
18      1
27      1
19      1
20      1
17      1
26      1
28      1
29      1
9       1
23      1
21      1
30      1
Name: id, dtype: int64

### Get result based on clustered data


#### Prepare data for extracting privileges from given label

In [65]:
user_data

,user_id,job,department,system_name,priv
0,314985,FIELD SERVICE,SERVICE TEAM MANAGER - NORTH,PE1CLNT110,PE1CLNT110:MM3120_DP:D_MATMAS_B+G
1,690192,FIELD SERVICE,TECHNICAL TRAINING AND VALIDATION MANAGER IC”,PE1CLNT110,PE1CLNT110:XX0000_SP:D_EVERYONE
2,2935844,CUSTOMER SERVICE,SACHBEARBEITER CUSTOMER SERVICE,QE2CLNT110,QE2CLNT110:MM3040_DP:D_INFOSYS
3,4020595,FIELD SERVICE,VALIDATION ENGINEER,QE2CLNT110,QE2CLNT110:LO8060_DP:M_LEAN_WM
4,4008660,FIELD SERVICE,SERVICE TECHNICIAN,PE1CLNT110,PE1CLNT110:PS8090_DP:D_PROJECT_MANAGER
...,...,...,...,...,...
21948,423013,SALES,SENIOR DIRECTOR LS SPECIAL PRODUCTS SALES,PE1CLNT110,PE1CLNT110:XX3120_DP:D_IDOC
21949,4021869,SALES,KEY ACCOUNT MANAGER BIOTECHNOLOGY,PE1CLNT110,PE1CLNT110:CO3030_DP:D_CO_DEFAULT
21950,2001018,CUSTOMER SERVICE,SECOND LEVEL TECHNICAL SUPPORT SPECIALIST,QE2CLNT110,QE2CLNT110:XX1600_DP:D_IDOC
21951,2335069,SALES,"MANAGING DIRECTOR, GETINGE GROUP, HONG KONG",QE2CLNT110,QE2CLNT110:PS8010_DP:D_PS


In [66]:
unique_user_data = user_data.groupby(['user_id', 'job', 'department']).describe().reset_index()

In [67]:
unique_user_data.columns

MultiIndex([(    'user_id',       ''),
            (        'job',       ''),
            ( 'department',       ''),
            ('system_name',  'count'),
            ('system_name', 'unique'),
            ('system_name',    'top'),
            ('system_name',   'freq'),
            (       'priv',  'count'),
            (       'priv', 'unique'),
            (       'priv',    'top'),
            (       'priv',   'freq')],
           )

In [68]:
unique_user_data

user_id               job                department system_name         \
                                                                count unique   
0          81             SALES  REGIONAL ACCOUNT MANAGER           7      3   
1         254     FIELD SERVICE     AREA MANAGER, SERVICE          17      2   
2         276     FIELD SERVICE  FIELD SERVICE TECHNICIAN           7      2   
3         295     FIELD SERVICE  FIELD SERVICE TECHNICIAN           6      2   
4         323     FIELD SERVICE            IWS SPECIALIST           9      2   
...       ...               ...                       ...         ...    ...   
2272  4023610     FIELD SERVICE  FIELD SERVICE TECHNICIAN          16      4   
2273  4023637  CUSTOMER SERVICE             SPECIALIST CS          19      2   
2274  4023638  CUSTOMER SERVICE      JUNIOR SPECIALIST CS          17      4   
2275  4023639  CUSTOMER SERVICE      JUNIOR SPECIALIST CS          14      4   
2276  4023652  CUSTOMER SERVICE      JUNIOR SPECIALIST CS          14      4   

                       priv                                                 \
             top freq count unique                                     top   
0     PE1CLNT110    3     7      7       QE2CLNT110:CS3180_DP:D_CS_DEFAULT   
1     PE1CLNT110    9    17     17         PE1CLNT110:LO5400_DP:M_DELIVERY   
2     PE1CLNT110    4     7      7       QE2CLNT110:MM3140_DP:D_MM_DEFAULT   
3     PE1CLNT110    3     6      6  QE2CLNT110:PS3280_DP:D_PROJECT_MANAGER   
4     PE1CLNT110    6     9      9               PE1CLNT110:LO3360_DP:D_LO   
...          ...  ...   ...    ...                                     ...   
2272  PE1CLNT110    8    16     16         QE2CLNT110:XX0000_SP:D_EVERYONE   
2273  PE1CLNT110   12    19     19               QE2CLNT110:LO3230_DP:D_LO   
2274  PE1CLNT110    8    17     17         PGWCLNT110:XX0000_SP:D_EVERYONE   
2275  PE1CLNT110    6    14     14               PE1CLNT110:MM6010_DP:M_PO   
2276  QE2CLNT110    7    14     14               QE2CLNT110:SD6900_DP:M_SD   

           
     freq  
0       1  
1       1  
2       1  
3       1  
4       1  
...   ...  
2272    1  
2273    1  
2274    1  
2275    1  
2276    1  

[2277 rows x 11 columns]

In [69]:
unique_user_data.drop(['system_name', 'priv'], axis=1, inplace=True)

/home/petar/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [70]:
unique_user_data.columns = ['user_id', 'job', 'department']

In [71]:
unique_user_data

,user_id,job,department
0,81,SALES,REGIONAL ACCOUNT MANAGER
1,254,FIELD SERVICE,"AREA MANAGER, SERVICE"
2,276,FIELD SERVICE,FIELD SERVICE TECHNICIAN
3,295,FIELD SERVICE,FIELD SERVICE TECHNICIAN
4,323,FIELD SERVICE,IWS SPECIALIST
...,...,...,...
2272,4023610,FIELD SERVICE,FIELD SERVICE TECHNICIAN
2273,4023637,CUSTOMER SERVICE,SPECIALIST CS
2274,4023638,CUSTOMER SERVICE,JUNIOR SPECIALIST CS
2275,4023639,CUSTOMER SERVICE,JUNIOR SPECIALIST CS


In [72]:
result_table = pd.concat([unique_user_data, user_privs_data.drop('user_id', axis=1)], axis=1)

In [73]:
result_table

,user_id,job,department,ASJAVAIDM:IDM_User,ASJAVAIDM:idm.user,DE2CLNT210:CO3050_DP:D_CO,DE2CLNT210:CO3050_DP:D_CO_DEFAULT,DE2CLNT210:CO3050_DP:D_PRODUCT_COST,DE2CLNT210:CO3160_DP:D_CO,DE2CLNT210:CO3160_DP:D_CO_DEFAULT,...,QG2CLNT110:GTS3220_DP:M_FI,QG2CLNT110:GTS3220_DP:M_WORKLIST,QG2CLNT110:XX0000_SP:D_EVERYONE,QGWCLNT110:XX0000_SP:D_EVERYONE,QGWCLNT110:XX0000_SP:D_EVERYONE_FIORI,QGWCLNT110:XX0000_SP:M_FIORI_CO,QGWCLNT110:XX0000_SP:M_FIORI_CS,QGWCLNT110:XX0000_SP:M_FIORI_TRV,QGWCLNT110:XX0000_SP:M_FIORI_TRV_APP,SE6CLNT110:ZSAP_ALL_ERP
0,81,SALES,REGIONAL ACCOUNT MANAGER,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,254,FIELD SERVICE,"AREA MANAGER, SERVICE",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,276,FIELD SERVICE,FIELD SERVICE TECHNICIAN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,295,FIELD SERVICE,FIELD SERVICE TECHNICIAN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,323,FIELD SERVICE,IWS SPECIALIST,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,4023610,FIELD SERVICE,FIELD SERVICE TECHNICIAN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2273,4023637,CUSTOMER SERVICE,SPECIALIST CS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2274,4023638,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2275,4023639,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
result_table.insert(3, "cluster_label", result.id)

In [75]:
result_table

,user_id,job,department,cluster_label,ASJAVAIDM:IDM_User,ASJAVAIDM:idm.user,DE2CLNT210:CO3050_DP:D_CO,DE2CLNT210:CO3050_DP:D_CO_DEFAULT,DE2CLNT210:CO3050_DP:D_PRODUCT_COST,DE2CLNT210:CO3160_DP:D_CO,...,QG2CLNT110:GTS3220_DP:M_FI,QG2CLNT110:GTS3220_DP:M_WORKLIST,QG2CLNT110:XX0000_SP:D_EVERYONE,QGWCLNT110:XX0000_SP:D_EVERYONE,QGWCLNT110:XX0000_SP:D_EVERYONE_FIORI,QGWCLNT110:XX0000_SP:M_FIORI_CO,QGWCLNT110:XX0000_SP:M_FIORI_CS,QGWCLNT110:XX0000_SP:M_FIORI_TRV,QGWCLNT110:XX0000_SP:M_FIORI_TRV_APP,SE6CLNT110:ZSAP_ALL_ERP
0,81,SALES,REGIONAL ACCOUNT MANAGER,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,254,FIELD SERVICE,"AREA MANAGER, SERVICE",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,276,FIELD SERVICE,FIELD SERVICE TECHNICIAN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,295,FIELD SERVICE,FIELD SERVICE TECHNICIAN,10,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,323,FIELD SERVICE,IWS SPECIALIST,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,4023610,FIELD SERVICE,FIELD SERVICE TECHNICIAN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2273,4023637,CUSTOMER SERVICE,SPECIALIST CS,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2274,4023638,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2275,4023639,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Extract all data from clusters with selected job and department in them

In [76]:
result_data = result_table.where((result_table.job == job) & (result_table.department == department))

In [77]:
result_data.dropna(inplace=True)

In [78]:
result_data = result_data.reset_index(drop=True)

In [79]:
result_data

,user_id,job,department,cluster_label,ASJAVAIDM:IDM_User,ASJAVAIDM:idm.user,DE2CLNT210:CO3050_DP:D_CO,DE2CLNT210:CO3050_DP:D_CO_DEFAULT,DE2CLNT210:CO3050_DP:D_PRODUCT_COST,DE2CLNT210:CO3160_DP:D_CO,...,QG2CLNT110:GTS3220_DP:M_FI,QG2CLNT110:GTS3220_DP:M_WORKLIST,QG2CLNT110:XX0000_SP:D_EVERYONE,QGWCLNT110:XX0000_SP:D_EVERYONE,QGWCLNT110:XX0000_SP:D_EVERYONE_FIORI,QGWCLNT110:XX0000_SP:M_FIORI_CO,QGWCLNT110:XX0000_SP:M_FIORI_CS,QGWCLNT110:XX0000_SP:M_FIORI_TRV,QGWCLNT110:XX0000_SP:M_FIORI_TRV_APP,SE6CLNT110:ZSAP_ALL_ERP
0,4017417.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,12.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4020599.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4020961.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4021375.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4021538.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4021540.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4021541.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4021746.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4021748.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,4022458.0,CUSTOMER SERVICE,JUNIOR SPECIALIST CS,12.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
result_data.cluster_label.value_counts()

6.0     6
14.0    3
0.0     3
12.0    2
8.0     1
4.0     1
18.0    1
11.0    1
7.0     1
Name: cluster_label, dtype: int64

#### Get  the best fitted cluster

In [81]:
index = 1
# if len(result_data.cluster_label.value_counts().index.tolist()) > 3:
#   index = 2

best_cluster_label = result_data.cluster_label.value_counts().index.tolist()[0: index]

In [82]:
best_cluster_label

[6.0]

In [83]:
privileges_counts = {}
users_count = 0
for c in best_cluster_label:
  current_cluster_label_data = result_data[result_data['cluster_label'] == c]
  current_cluster_label_data.reset_index(drop=True, inplace=True)
  users_count += current_cluster_label_data.shape[0]
  if 'index' in current_cluster_label_data.columns:
    current_cluster_label_data = current_cluster_label_data.drop('index', axis=1)
  for pr in current_cluster_label_data.columns[4:]:
    if 1 in current_cluster_label_data[pr].value_counts().index.tolist():
      if pr not in privileges_counts.keys():
        privileges_counts[pr] = 0  
      privileges_counts[pr] += current_cluster_label_data[pr].value_counts()[1]

In [84]:
users_count

6

In [85]:
#privileges_counts = dict(sorted(privileges_counts.items(),key= lambda x:x[1]))
sorted_privileges_counts = sorted(privileges_counts.items(), key=lambda x: x[1], reverse=True)
sorted_privileges_counts

[('QE2CLNT110:XX0000_SP:D_EVERYONE', 5),
 ('QE2CLNT110:MM6900_DP:D_PO_MESSAGES', 3),
 ('P11CLNT110:XX0025_DP:D_COMPANY', 2),
 ('PE1CLNT110:CO6900_DP:D_CO_DEFAULT', 2),
 ('PE1CLNT110:CS6900_DP:D_CS_STATISTIC', 2),
 ('PE1CLNT110:PP6900_DP:D_PP_DEFAULT', 2),
 ('PO1CLNT110:SM0000_SP:D_EVERYONE_PO1', 2),
 ('P11CLNT110:SD0025_DP:D_SALES_STATISTIC', 1),
 ('PE1CLNT110:CO3140_DP:D_CO', 1),
 ('PE1CLNT110:CO3140_DP:D_CO_DEFAULT', 1),
 ('PE1CLNT110:CS3280_DP:D_CS_DEFAULT', 1),
 ('PE1CLNT110:CS5400_DP:D_CS_DEFAULT', 1),
 ('PE1CLNT110:CS5400_DP:D_CS_STATISTIC', 1),
 ('PE1CLNT110:LO3270_DP:D_LO', 1),
 ('PE1CLNT110:LO5400_DP:D_LO', 1),
 ('PE1CLNT110:LO5400_DP:D_LO_DEFAULT', 1),
 ('PE1CLNT110:LO6900_DP:D_LO_DEFAULT', 1),
 ('PE1CLNT110:MM3140_DP:D_MM_DEFAULT', 1),
 ('PE1CLNT110:MM3270_DP:D_MM_DEFAULT', 1),
 ('PE1CLNT110:MM3270_DP:D_PURCHASING', 1),
 ('PE1CLNT110:MM3280_DP:D_STOCKS', 1),
 ('PE1CLNT110:MM5400_DP:D_PO_MESSAGES', 1),
 ('PE1CLNT110:MM5400_DP:D_PURCHASING', 1),
 ('PE1CLNT110:MM6900_DP:D_MATMA

In [86]:
filtered_privileges_counts = list(filter(lambda x: x[1] > users_count*0.4, sorted_privileges_counts))

#### Filter the most relevant privileges from that cluster

In [87]:
filtered_privileges_counts

[('QE2CLNT110:XX0000_SP:D_EVERYONE', 5),
 ('QE2CLNT110:MM6900_DP:D_PO_MESSAGES', 3)]

In [88]:
suggested_privileges = [v[0] for v in filtered_privileges_counts]

In [89]:
suggested_privileges

['QE2CLNT110:XX0000_SP:D_EVERYONE', 'QE2CLNT110:MM6900_DP:D_PO_MESSAGES']

### Applying PCA to the model data

To reduce dimentionality of the dataset we are going to use PCA from sckitlearn

In [90]:
#pca = PCA()

In [91]:
#new_transformed_privs = pca.fit_transform(user_privs_data.drop('user_id', axis=1))

In [92]:
#new_transformed_privs.shape

In [93]:
#transformed_user_privs_data = pd.DataFrame(new_transformed_privs, columns=user_privs_data.drop('user_id', axis=1).columns)

In [94]:
#transformed_user_privs_data

In [95]:
#pca.explained_variance_ratio_.shape

In [96]:
#user_privs_data.drop('user_id', axis=1).columns.shape

In [97]:
#variance_explained = pd.DataFrame([pca.explained_variance_ratio_], columns=user_privs_data.drop('user_id', axis=1).columns)

In [98]:
#variance_explained

In [99]:
#fit_pca_data = agg_clustering.fit(transformed_user_privs_data)

In [100]:
#fit_pca_data.n_clusters_

In [101]:
#result = pd.DataFrame(fit_pca_data.labels_, columns=['id'])

In [102]:
#result

In [103]:
#result.id.value_counts()